In [7]:
import numpy as np
from scipy.io import loadmat
from scipy.signal.windows import tukey

In [5]:
struct = loadmat("temp.mat")
arr = struct["arr"]
arr[0, 0]

(-0.5320114-1.0667014j)

In [14]:
n_x, n_y = arr.shape
ARG = {"phase_filter_width": 3}

# Grab the 2D slice of the 4D array
slice_data = arr.copy()

# Apply 1D FFT to the 2D slice
for k_dim in range(2):
    slice_data = np.fft.ifftshift(
        np.fft.ifft(
            np.fft.ifftshift(slice_data, axes=[k_dim]),
            n=None,
            axis=k_dim,
        ),
        axes=[k_dim],
    )

# Apply Tukey window to the filtered 2D slice
# TODO: Verify that this works.
# tmp = bsxfun(@times,tmp,reshape(tukeywin(n_y,1).^phase_filter_width,[1 n_y]));
tukey_window = tukey(n_y, 1) ** ARG["phase_filter_width"]
tukey_window_reshaped = tukey_window.reshape(1, n_y)
slice_data = slice_data * tukey_window_reshaped

# tmp = bsxfun(@times,tmp,reshape(tukeywin(n_x,1).^phase_filter_width,[n_x 1]));
tukey_window = tukey(n_x, 1).T ** ARG["phase_filter_width"]
tukey_window_reshaped = tukey_window.reshape(n_x, 1)
slice_data = slice_data * tukey_window_reshaped

# Apply 1D IFFT to the filtered 2D slice and store in the 4D array
for k_dim in range(2):
    slice_data = np.fft.fftshift(
        np.fft.fft(
            np.fft.fftshift(slice_data, axes=[k_dim]),
            n=None,
            axis=k_dim,
        ),
        axes=[k_dim],
    )

In [17]:
slice_data[4, 5]

(-0.0281431665240858+0.08233896479817929j)

In [21]:
struct = loadmat("temp2.mat")
tmp = struct["tmp"]
tmp[0, 0]

(-0.1939142-0.16348928j)

In [22]:
np.allclose(tmp, slice_data)

True